## Run Diamond

Runs `diamond` with given reference files and input. 
Needs diamond tool on hdfs which loaded before running.

In [ ]:
import os
import subprocess
import sys
import stat
import traceback

from hops import hdfs
from pyspark import SparkContext
import utils

sc = SparkContext.getOrCreate()


#### load arguments

In [ ]:


args_full=utils.load_arguments(sys.argv)

OUTPUT_DATASET=args_full[utils.OUTPUT_DATASET]
INPUT_ROOT_PATH=args_full[utils.INPUT_ROOT_PATH]
RUN_FOLDER=args_full[utils.RUN_FOLDER]
WORK_PATH=os.path.join(OUTPUT_DATASET, RUN_FOLDER)
args=args_full[utils.KEY_DIAMOND]
# check of input and output root override
if args_full.get(utils.INPUT_OVERRIDE):
    INPUT_ROOT=args_full.get(utils.INPUT_OVERRIDE)
else :
   INPUT_ROOT=os.path.join(WORK_PATH,args['INPUT_ROOT'])
if args_full.get(utils.OUTPUT_OVERRIDE):
    OUTPUT_ROOT=args_full.get(utils.OUTPUT_OVERRIDE)
else:
    OUTPUT_ROOT=os.path.join(WORK_PATH,args['OUTPUT_ROOT'])





REFERENCE_FASTA_PATH = args['REFERENCE_FASTA']
DIAMOND_RUN= args['DIAMOND_RUN']
TOP = args['TOP']
COMPRESS = args['COMPRESS']
OUTPUT_FORMAT=args['OUTPUT_FORMAT']
OUTPUT_PREFIX=args['OUTPUT_PREFIX']
ADVANCE_TUNING = args['ADVANCED_TUNING']
LOG_DIR=args['LOGS_ROOT']
DIAMOND_TOOL=args['DIAMOND_TOOL']

inputRef= os.path.split(REFERENCE_FASTA_PATH)[1]
DIAMOND_PREFIX='./diamond'
SPACE=utils.SPACE

#### Functions to load diamond and to make diamond db

In [ ]:
# load diamond scripts from hdfs
def install_diamond(diamond_path):
    hdfs.copy_to_local(diamond_path)
    st = os.stat(DIAMOND_PREFIX)
    os.chmod(DIAMOND_PREFIX, st.st_mode | stat.S_IEXEC)


### MAKE DIAMOND DB
def make_diamond_db(reference_path):
    
    install_diamond(DIAMOND_TOOL)
    diamond_DB=os.path.split(reference_path)[1]+'_DB.dmnd'
    hdfs.copy_to_local(reference_path)
    params={ 'makedb --in': inputRef, '-d':diamond_DB }
    cmd_DB=utils.build_command(DIAMOND_PREFIX, params)
      
    status=subprocess.run(cmd_DB.split(' '),stdout=subprocess.PIPE,check=True)

    if status.returncode==0:
        hdfs_path=os.path.split(reference_path)[0]
        hdfs.copy_to_hdfs(diamond_DB,hdfs_path,overwrite=True)
        
        return os.path.join(hdfs_path,diamond_DB)

    return None




#### Map function running diamond on single file

In [ ]:



def apply_diamond(file,diamondDB_path):
    """
    Runs diamond via subprcess on single file.
    Output and diamond log is copied back to hdfs.

    :param file:input fastq file:
    :param diamondDB_path:
    :return:
    """

    #install diamond
    if not os.path.exists(DIAMOND_PREFIX):
        install_diamond(DIAMOND_TOOL)
    
    # download diamond DB file
    diamond_DB=os.path.split(diamondDB_path)[1]
    if not os.path.exists(diamond_DB):
        hdfs.copy_to_local(diamondDB_path)
        
    # download input file
    hdfs.copy_to_local(file, overwrite=True)
    file=os.path.basename(file)
    filename=os.path.splitext(os.path.splitext(file)[0])[0] # split file name without extension    
            
    outfile=OUTPUT_PREFIX+filename     

    if ADVANCE_TUNING:
        advanced_params = str(COMPRESS)+SPACE+ADVANCE_TUNING
    else :
        advanced_params = str(COMPRESS)
        
    parameters = { '-d': diamond_DB, '-q':file, '-o':outfile, '--top': TOP, '--outfmt':OUTPUT_FORMAT, '--compress': advanced_params}
    cmd=utils.build_command(DIAMOND_PREFIX+SPACE+DIAMOND_RUN,parameters)
    print('INFO: Running diamond with command:', cmd)
       
    log_file=os.path.splitext(outfile)[0]+'.txt'
    try:
        with open(log_file, "w") as f:
            execStatus=subprocess.run(cmd.split(' '),stdout=f,stderr=f,check=True)

        hdfs.copy_to_hdfs(log_file, LOG_DIR, overwrite=True)
        if COMPRESS:
            target=outfile+'.gz'
        else:
            target=outfile

        if execStatus.returncode==0 and os.path.exists(target):
            hdfs.copy_to_hdfs(target,OUTPUT_ROOT,overwrite=True)
            os.remove(target)

        return target
    except subprocess.SubprocessError:
        traceback.print_exc()
        return False
    finally:
        parameters.clear()
        os.remove(file)
        if os.path.exists(log_file):
            os.remove(log_file)

    

#### Run

* first make diamond db
* load input hdfs file paths
* make spark rdd and map function to run parallel


In [ ]:
# make diamond DB
diamondDB_path=make_diamond_db(REFERENCE_FASTA_PATH)    

# load input data
inputFiles=utils.load_file_names(INPUT_ROOT)
# run
final=sc.parallelize(inputFiles,sc.getConf().get("spark.executor.instances")).map(lambda x: apply_diamond(x,diamondDB_path) ).collect()


